<a href="https://colab.research.google.com/github/trucni2005/trucni2005/blob/main/GKTTNT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#minimax với random
from typing import MutableMapping
import random
from termcolor import colored
from IPython.display import clear_output
import time
# K, k: Vua
# Q, q: Hậu
# R, r: Xe
# B, b: Tượng
# N, n: Mã
# P, p: Tốt
liR = [(1,0),(2,0),(3,0),(4,0),(5,0),(6,0),(7,0)]
liL = [(-1,0),(-2,0),(-3,0),(-4,0),(-5,0),(-6,0),(-7,0)]
liD = [(0,1),(0,2),(0,3),(0,4),(0,5),(0,6),(0,7)]
liU = [(0,-1),(0,-2),(0,-3),(0,-4),(0,-5),(0,-6),(0,-7)]
liRD = [(1,1),(2,2),(3,3),(4,4),(5,5),(6,6),(7,7)]
liLU = [(-1,-1),(-2,-2),(-3,-3),(-4,-4),(-5,-5),(-6,-6),(-7,-7)]
liRU = [(1,-1),(2,-2),(3,-3),(4,-4),(5,-5),(6,-6),(7,-7)]
liLD = [(-1,1),(-2,2),(-3,3),(-4,4),(-5,5),(-6,6),(-7,7)]
liN = [(1,2),(-1,2),(1,-2),(-1,-2),(2,1),(-2,1),(2,-1),(-2,-1)]
liK = [(0,1),(1,0),(0,-1),(-1,0),(1,1),(-1,1),(1,-1),(-1,-1)]

board = [['r','n','b','q','k','b','n','r'],
         ['p','p','p','p','p','p','p','p'],
         ['_','_','_','_','_','_','_','_'],
         ['_','_','_','_','_','_','_','_'],
         ['_','_','_','_','_','_','_','_'],
         ['_','_','_','_','_','_','_','_'],
         ['P','P','P','P','P','P','P','P'],
         ['R','N','B','Q','K','B','N','R']]

# kiểm tra có thể đi được hay không (Xe, Tượng, Hậu)
def Go(x,y,l,isContinue=False,canFight=True,needFight=False):
  li = []
  for dx,dy in l:
    x1 = x+dx
    y1 = y+dy
    if x1<0 or x1>=8 or y1<0 or y1>=8: 
      if not isContinue: 
        break 
      else: 
        continue
    if board[y1][x1] != '_':
      if canFight and board[y][x].islower() != board[y1][x1].islower():
        li.append((x1,y1))
      if not isContinue: break 
      else: continue
    if not needFight:
      li.append((x1,y1))
  return li


# các nước có thể đi của quân Xe
def Rr(x,y):
  return Go(x,y,liR) + Go(x,y,liL) + Go(x,y,liU) + Go(x,y,liD)

# các nước có thể đi của quân Tượng
def Bb(x,y):
  return Go(x,y,liRD) + Go(x,y,liLU) + Go(x,y,liRU) + Go(x,y,liLD)

# các nước có thể đi của quân Hậu
def Qq(x,y):
  return Rr(x,y) + Bb(x,y)

# các nước có thể đi của quân Mã
def Nn(x,y):
  return Go(x,y,liN,isContinue=True)

# các nước có thể đi của quân Vua
def Kk(x,y):
  return Go(x,y,liK,isContinue=True)

# các nước có thể đi của quân Tốt
def Pp(x,y):
  li = []
  dx = 1
  if not board[y][x].islower():
    dx = -1
  # nước có thể đi
  if y==1 or y==6:
    li = li + Go(x,y,[(0,dx),(0,2*dx)],canFight=False)
  else:
    li = li + Go(x,y,[(0,dx)], canFight=False)
  # nước có thể ăn (tốt ăn chéo)
  li = li + Go(x,y,[(-1,dx),(1,dx)],needFight=True)
  return li

# x, y là tọa độ quân cờ
# islower lượt của quân thường (True) hay quân hoa (False)
# return danh sách tọa độ các nước có thể di chuyển tới
def CanGo(x,y,islower):
  if board[y][x].islower() != islower:
    return []
  if (board[y][x] in "Rr"):
    return Rr(x,y)
  if (board[y][x] in "Nn"):
   return Nn(x,y)
  if (board[y][x] in "Bb"):
    return Bb(x,y)
  if (board[y][x] in "Qq"):
    return Qq(x,y)
  if (board[y][x] in "Kk"):
    return Kk(x,y)
  if (board[y][x] in "Pp"):
    return Pp(x,y)
  return []


def isFinish(board):
  U = False
  L = False
  for _ in board:
    U = U or ('K' in _)
    L = L or ('k' in _)
  return not (U and L)

# return tất cả các nước có thể di chuyển (y,x) -> (y1,x1)
def CanGoList(board,islower):
  li = []
  for y in range(8):
    for x in range(8):
      l = CanGo(x,y,islower)
      for x1,y1 in l:
        li = li + [(y,x,y1,x1)]
  return li

def CPURandomTurn(board,islower):
  #######################################################
  #  Có thể bỏ time.sleep(1) để xem kết quả nhanh hơn   #
  ####################################################### 
  li = CanGoList(board,islower)
  return random.choice(li)

#minimax root
def CPUMiniMaxTurn(board,islower,depth = 2):
  #######################################################
  #  Nên dùng Minimax với độ sâu từ 2 đến 4             #
  #######################################################
  #li là list có thể đi của lượt
  li = CanGoList(board,islower)
  Max = -1000
  for y,x,y1,x1 in CanGoList(board,islower):
    child = [_[:] for _ in board]
    #di chuyển quân cờ tọa độ y,x đến tọa độ y1,x1
    child[y1][x1] = child[y][x]
    child[y][x] = '_'
    vl = Minimax(child,depth-1,islower,not islower,-1000,1000)
    #nếu max < value hoặc max = value nhưng random choice = 0 thì thay max = value, r = tọa độ khác, return r
    if Max < vl or (Max == vl and random.choice([0,1])==0):
      Max = vl
      r = (y,x,y1,x1)
  return r


# Board: Bàn cờ hiện tại
# islower: lượt của quân viết thường (True) hay quân viết hoa (False)
# return: giá trị của bàn cờ đối với quân viết thường

# K, k: Vua
# Q, q: Hậu
# R, r: Xe
# B, b: Tượng
# N, n: Mã
# P, p: Tốt

def Check(g):
  if (g=="_"):
    return 0;
  if (g=="k"):
    return -99
  if (g=="q"):
    return -15
  if (g=="b"):
    return -6
  if (g=="n"):
    return -3
  if (g=="r"):
    return -9
  if (g=="p"):
    return -1
  if (g=="K"):
    return 99
  if (g=="Q"):
    return 15
  if (g=="B"):
    return 6
  if (g=="N"):
    return 3
  if (g=="R"):
    return 9
  if (g=="P"):
    return 1
  
#evaluateBoard
def value(board,islower):
  vl = 0
  ######################################################
  #  Cần định nghĩa lại hàm value                      #
  #  Gợi ý:                                            #
  #       1. Hiệu số quân cờ 2 bên                     #
  #       2. Vị trí quân cờ trên bàn cờ                #
  #       3. Tùy suy nghĩ của các em                   #
  ######################################################
  for i in range(8):
    for j in range(8):
      vl = vl + Check(board[i][j])
  return vl;


# node là thế cờ hiện tại
# depth là độ sâu
# Pmax là player cần tìm Max
# Pnow là player hiện tại

#minimax
def Minimax(node,depth,Pmax,Pnow,alpha,beta):
  if isFinish(node) or depth ==0:
    return value(node,Pmax)
  #islower == not islower
  if Pmax == Pnow:
    Max = -1000
    for y,x,y1,x1 in CanGoList(node,Pnow):
      child = [_[:] for _ in node]
      child[y1][x1] = child[y][x]
      child[y][x] = '_'
      Max = max(Max,Minimax(child,depth-1,Pmax,not Pnow,alpha,beta))
      alpha = max(alpha, Max);
      if (beta <= alpha):
        return Max;
    return Max
  else:
    Min = 1000
    for y,x,y1,x1 in CanGoList(node,Pnow):
      child = [_[:] for _ in node]
      child[y1][x1] = child[y][x]
      child[y][x] = '_'
      Min = min(Min,Minimax(child,depth-1,Pmax,not Pnow,alpha,beta))
      beta = min(beta, Min);
      if (beta <= alpha):
        return Min;
    return Min

#in bàn cờ
#quân thường đi trước
def printBoard(board):
  print("+",*range(8),"+")

  for i in range(8):
    print(i,end=" ")
    for j in range(8):
      if board[i][j] == '_':
        print(board[i][j],end=" ")
      elif board[i][j].islower():
        print(colored(board[i][j],'red'),end=" ")
      else:
        print(colored(board[i][j],'blue'),end=" ")
    print(i,) 

  print("+",*range(8),"+")
  print()

Random = True
MiniMax = False
Now = Random

while not isFinish(board):
  clear_output()
  printBoard(board)
  #now = random
  print("----",Now,"turn","----")
  #nếu now = minimax = false
  if Now == MiniMax:
    #tính theo minimax turn
    y,x,y1,x1 = CPUMiniMaxTurn(board,MiniMax)
    board[y1][x1] = board[y][x]
    board[y][x] = '_'
    #set now = random
    Now = Random
    #in thay đổi
    print(y,x,y1,x1)
  else:
    #tính theo random turn = true
    y,x,y1,x1 = CPURandomTurn(board,Random)
    board[y1][x1] = board[y][x]
    board[y][x] = '_'
    #in thay đổi
    print(y,x,y1,x1)
    #set now = mini max turn
    Now = MiniMax

printBoard(board)
if Now:
  print("Minimax Won")
else:
  print("Random Won")

+ 0 1 2 3 4 5 6 7 +
0 _ _ _ P _ b _ _ 0
1 _ _ _ _ _ k _ _ 1
2 r _ n N _ _ _ _ 2
3 p _ p _ _ _ _ _ 3
4 _ _ _ _ _ _ R _ 4
5 _ _ _ _ _ N _ _ 5
6 P P P P P P P _ 6
7 R _ B Q K B _ _ 7
+ 0 1 2 3 4 5 6 7 +

---- False turn ----
2 3 1 5
+ 0 1 2 3 4 5 6 7 +
0 _ _ _ P _ b _ _ 0
1 _ _ _ _ _ N _ _ 1
2 r _ n _ _ _ _ _ 2
3 p _ p _ _ _ _ _ 3
4 _ _ _ _ _ _ R _ 4
5 _ _ _ _ _ N _ _ 5
6 P P P P P P P _ 6
7 R _ B Q K B _ _ 7
+ 0 1 2 3 4 5 6 7 +

Minimax Won


In [ ]:

from typing import MutableMapping
import random
from termcolor import colored
from IPython.display import clear_output
import time
# K, k: Vua
# Q, q: Hậu
# R, r: Xe
# B, b: Tượng
# N, n: Mã
# P, p: Tốt
liR = [(1,0),(2,0),(3,0),(4,0),(5,0),(6,0),(7,0)]
liL = [(-1,0),(-2,0),(-3,0),(-4,0),(-5,0),(-6,0),(-7,0)]
liD = [(0,1),(0,2),(0,3),(0,4),(0,5),(0,6),(0,7)]
liU = [(0,-1),(0,-2),(0,-3),(0,-4),(0,-5),(0,-6),(0,-7)]
liRD = [(1,1),(2,2),(3,3),(4,4),(5,5),(6,6),(7,7)]
liLU = [(-1,-1),(-2,-2),(-3,-3),(-4,-4),(-5,-5),(-6,-6),(-7,-7)]
liRU = [(1,-1),(2,-2),(3,-3),(4,-4),(5,-5),(6,-6),(7,-7)]
liLD = [(-1,1),(-2,2),(-3,3),(-4,4),(-5,5),(-6,6),(-7,7)]
liN = [(1,2),(-1,2),(1,-2),(-1,-2),(2,1),(-2,1),(2,-1),(-2,-1)]
liK = [(0,1),(1,0),(0,-1),(-1,0),(1,1),(-1,1),(1,-1),(-1,-1)]

board = [['r','n','b','q','k','b','n','r'],
         ['p','p','p','p','p','p','p','p'],
         ['_','_','_','_','_','_','_','_'],
         ['_','_','_','_','_','_','_','_'],
         ['_','_','_','_','_','_','_','_'],
         ['_','_','_','_','_','_','_','_'],
         ['P','P','P','P','P','P','P','P'],
         ['R','N','B','Q','K','B','N','R']]

# kiểm tra có thể đi được hay không (Xe, Tượng, Hậu)
def Go(x,y,l,isContinue=False,canFight=True,needFight=False):
  li = []
  for dx,dy in l:
    x1 = x+dx
    y1 = y+dy
    if x1<0 or x1>=8 or y1<0 or y1>=8: 
      if not isContinue: 
        break 
      else: 
        continue
    if board[y1][x1] != '_':
      if canFight and board[y][x].islower() != board[y1][x1].islower():
        li.append((x1,y1))
      if not isContinue: break 
      else: continue
    if not needFight:
      li.append((x1,y1))
  return li


# các nước có thể đi của quân Xe
def Rr(x,y):
  return Go(x,y,liR) + Go(x,y,liL) + Go(x,y,liU) + Go(x,y,liD)

# các nước có thể đi của quân Tượng
def Bb(x,y):
  return Go(x,y,liRD) + Go(x,y,liLU) + Go(x,y,liRU) + Go(x,y,liLD)

# các nước có thể đi của quân Hậu
def Qq(x,y):
  return Rr(x,y) + Bb(x,y)

# các nước có thể đi của quân Mã
def Nn(x,y):
  return Go(x,y,liN,isContinue=True)

# các nước có thể đi của quân Vua
def Kk(x,y):
  return Go(x,y,liK,isContinue=True)

# các nước có thể đi của quân Tốt
def Pp(x,y):
  li = []
  dx = 1
  if not board[y][x].islower():
    dx = -1
  # nước có thể đi
  if y==1 or y==6:
    li = li + Go(x,y,[(0,dx),(0,2*dx)],canFight=False)
  else:
    li = li + Go(x,y,[(0,dx)], canFight=False)
  # nước có thể ăn (tốt ăn chéo)
  li = li + Go(x,y,[(-1,dx),(1,dx)],needFight=True)
  return li

# x, y là tọa độ quân cờ
# islower lượt của quân thường (True) hay quân hoa (False)
# return danh sách tọa độ các nước có thể di chuyển tới
def CanGo(x,y,islower):
  if board[y][x].islower() != islower:
    return []
  if (board[y][x] in "Rr"):
    return Rr(x,y)
  if (board[y][x] in "Nn"):
   return Nn(x,y)
  if (board[y][x] in "Bb"):
    return Bb(x,y)
  if (board[y][x] in "Qq"):
    return Qq(x,y)
  if (board[y][x] in "Kk"):
    return Kk(x,y)
  if (board[y][x] in "Pp"):
    return Pp(x,y)
  return []


def isFinish(board):
  U = False
  L = False
  for _ in board:
    U = U or ('K' in _)
    L = L or ('k' in _)
  return not (U and L)

# return tất cả các nước có thể di chuyển (y,x) -> (y1,x1)
def CanGoList(board,islower):
  li = []
  for y in range(8):
    for x in range(8):
      l = CanGo(x,y,islower)
      for x1,y1 in l:
        li = li + [(y,x,y1,x1)]
  return li

def CPURandomTurn(board,islower,depth = 3):
  #######################################################
  #  Có thể bỏ time.sleep(1) để xem kết quả nhanh hơn   #
  ####################################################### 
  li = CanGoList(board,islower)
  Max = -1000
  for y,x,y1,x1 in CanGoList(board,islower):
    child = [_[:] for _ in board]
    #di chuyển quân cờ tọa độ y,x đến tọa độ y1,x1
    child[y1][x1] = child[y][x]
    child[y][x] = '_'
    vl = Minimax(child,depth-1,islower,not islower,0,0)
    #nếu max < value hoặc max = value nhưng random choice = 0 thì thay max = value, r = tọa độ khác, return r
    if Max < vl or (Max == vl and random.choice([0,1])==0):
      Max = vl
      r = (y,x,y1,x1)
  return r

#minimax root
def CPUMiniMaxTurn(board,islower,depth = 3):
  #######################################################
  #  Nên dùng Minimax với độ sâu từ 2 đến 4             #
  #######################################################
  #li là list có thể đi của lượt
  li = CanGoList(board,islower)
  Max = -1000
  for y,x,y1,x1 in CanGoList(board,islower):
    child = [_[:] for _ in board]
    #di chuyển quân cờ tọa độ y,x đến tọa độ y1,x1
    child[y1][x1] = child[y][x]
    child[y][x] = '_'
    vl = Minimax(child,depth-1,islower,not islower,-1000, 1000)
    #nếu max < value hoặc max = value nhưng random choice = 0 thì thay max = value, r = tọa độ khác, return r
    if Max < vl or (Max == vl and random.choice([0,1])==0):
      Max = vl
      r = (y,x,y1,x1)
  return r


# Board: Bàn cờ hiện tại
# islower: lượt của quân viết thường (True) hay quân viết hoa (False)
# return: giá trị của bàn cờ đối với quân viết thường

# K, k: Vua
# Q, q: Hậu
# R, r: Xe
# B, b: Tượng
# N, n: Mã
# P, p: Tốt

def Check1(g):
  if (g=="_"):
    return 0;
  if (g=="k"):
    return 99
  if (g=="q"):
    return 15
  if (g=="b"):
    return 3
  if (g=="n"):
    return 3
  if (g=="r"):
    return 5
  if (g=="p"):
    return 1
  if (g=="K"):
    return -99
  if (g=="Q"):
    return -9
  if (g=="B"):
    return -3
  if (g=="N"):
    return -3
  if (g=="R"):
    return -5
  if (g=="P"):
    return -1

def Check(g):
  if (g=="_"):
    return 0;
  if (g=="k"):
    return -99
  if (g=="q"):
    return -15
  if (g=="b"):
    return -6
  if (g=="n"):
    return -3
  if (g=="r"):
    return -9
  if (g=="p"):
    return -1
  if (g=="K"):
    return 99
  if (g=="Q"):
    return 15
  if (g=="B"):
    return 6
  if (g=="N"):
    return 3
  if (g=="R"):
    return 9
  if (g=="P"):
    return 1

#evaluateBoard
def value(board,islower):
  vl = 0
  ######################################################
  #  Cần định nghĩa lại hàm value                      #
  #  Gợi ý:                                            #
  #       1. Hiệu số quân cờ 2 bên                     #
  #       2. Vị trí quân cờ trên bàn cờ                #
  #       3. Tùy suy nghĩ của các em                   #
  ######################################################
  if (islower==False):
    for i in range(8):
      for j in range(8):
        vl = vl + Check(board[i][j])
  else:
   for i in range(8):
      for j in range(8):
        vl = vl + Check1(board[i][j])
  return vl;


# node là thế cờ hiện tại
# depth là độ sâu
# Pmax là player cần tìm Max
# Pnow là player hiện tại

#minimax
def Minimax(node,depth,Pmax,Pnow,alpha,beta):
  if isFinish(node) or depth ==0:
    return value(node,Pmax)
  #islower == not islower
  if Pmax == Pnow:
    Max = -1000
    for y,x,y1,x1 in CanGoList(node,Pnow):
      child = [_[:] for _ in node]
      child[y1][x1] = child[y][x]
      child[y][x] = '_'
      Max = max(Max,Minimax(child,depth-1,Pmax,not Pnow,alpha,beta))
      alpha = max(alpha, Max);
      if (beta <= alpha):
        return Max;
    return Max
  else:
    Min = 1000
    for y,x,y1,x1 in CanGoList(node,Pnow):
      child = [_[:] for _ in node]
      child[y1][x1] = child[y][x]
      child[y][x] = '_'
      Min = min(Min,Minimax(child,depth-1,Pmax,not Pnow,alpha,beta))
      beta = min(beta, Min);
      if (beta <= alpha):
        return Min;
    return Min

#in bàn cờ
#quân thường đi trước
def printBoard(board):
  print("+",*range(8),"+")

  for i in range(8):
    print(i,end=" ")
    for j in range(8):
      if board[i][j] == '_':
        print(board[i][j],end=" ")
      elif board[i][j].islower():
        print(colored(board[i][j],'red'),end=" ")
      else:
        print(colored(board[i][j],'blue'),end=" ")
    print(i,) 

  print("+",*range(8),"+")
  print()

Random = True
MiniMax = False
Now = Random

while not isFinish(board):
  clear_output()
  printBoard(board)
  #now = random
  print("----",Now,"turn","----")
  #nếu now = minimax = false
  if Now == MiniMax:
    #tính theo minimax turn
    y,x,y1,x1 = CPUMiniMaxTurn(board,MiniMax)
    board[y1][x1] = board[y][x]
    board[y][x] = '_'
    #set now = random
    Now = Random
    #in thay đổi
    print(y,x,y1,x1)
  else:
    #tính theo random turn = true
    y,x,y1,x1 = CPURandomTurn(board,Random)
    board[y1][x1] = board[y][x]
    board[y][x] = '_'
    #in thay đổi
    print(y,x,y1,x1)
    #set now = mini max turn
    Now = MiniMax

printBoard(board)
if Now:
  print("Minimax Won")
else:
  print("Random Won")

+ 0 1 2 3 4 5 6 7 +
0 _ n b _ _ _ _ _ 0
1 _ _ _ _ _ _ _ k 1
2 _ P _ _ _ _ K _ 2
3 _ _ _ _ _ _ _ _ 3
4 P _ _ _ _ _ _ _ 4
5 _ _ _ _ _ _ _ _ 5
6 _ _ _ _ _ _ _ _ 6
7 _ R _ _ _ _ _ _ 7
+ 0 1 2 3 4 5 6 7 +

---- False turn ----
2 6 1 7
+ 0 1 2 3 4 5 6 7 +
0 _ n b _ _ _ _ _ 0
1 _ _ _ _ _ _ _ K 1
2 _ P _ _ _ _ _ _ 2
3 _ _ _ _ _ _ _ _ 3
4 P _ _ _ _ _ _ _ 4
5 _ _ _ _ _ _ _ _ 5
6 _ _ _ _ _ _ _ _ 6
7 _ R _ _ _ _ _ _ 7
+ 0 1 2 3 4 5 6 7 +

Minimax Won
